confusion matrix :
    http://gcucurull.github.io/caffe/python/deep-learning/2016/06/29/caffe-confusion-matrix/
    https://github.com/gcucurull/caffe-conf-matrix
FCN :
    score :
        https://github.com/shelhamer/fcn.berkeleyvision.org/blob/master/score.py
    infer :
        https://github.com/shelhamer/fcn.berkeleyvision.org/blob/master/infer.py

In [1]:
from __future__ import division
from __future__ import print_function
import caffe
import numpy as np
import os
import sys
from datetime import datetime
from PIL import Image
from IPython.display import Image
from IPython.display import clear_output
import time

In [2]:
def log_progress(sequence, title='', every=1, size=None, end_msg='', removeAtTheEnd=False):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    if isinstance(sequence, dict):
                        val = ''
                    else:
                        val=sequence[index-1]
                            
                    label.value = u'{title} : {value} ({index} / {size})'.format(
                        title=title,
                        value=val,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = end_msg
        if removeAtTheEnd:
            box.close()
    

In [3]:
def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n**2).reshape(n, n)

In [4]:
import _init_paths
import caffe
#import jg_layers.jg_input_voc_layers

In [5]:
def evaluate(results):
    
    hist_cumul = 0*results[0]
    for hist in results:
        hist_cumul+=hist
    result = {}
    

    acc = np.diag(hist_cumul).sum() / hist_cumul.sum()
    # per-class accuracy
    pc_acc = np.diag(hist_cumul) / hist_cumul.sum(1)
    mean_accuracy= np.nanmean(pc_acc)
    # per-class IU
    iu = np.diag(hist_cumul) / (hist_cumul.sum(1) + hist_cumul.sum(0) - np.diag(hist_cumul))
    mean_iu= np.nanmean(iu)
    freq = hist_cumul.sum(1) / hist_cumul.sum()
    freq_weighted_iu=(freq[freq > 0] * iu[freq > 0]).sum()
    
    print("Accuracy :",acc)
    print("per-class accuracy :",mean_accuracy)
    print("per-class IU :",mean_iu)
    print("per-class IU (frequency weighted) :",freq_weighted_iu)
    
    return acc

In [6]:
ICIPmodels = '/home/jguerry/workspace/ICIPmodels/'
prototxt_path = ICIPmodels+'jg_eval_gray.prototxt'
#prototxt_path = ICIPmodels+'jg_eval.prototxt'
caffe_gpu = True
if not caffe_gpu:
    caffe.set_mode_cpu()
else:
    caffe.set_mode_gpu()
    caffe.set_device(0);

In [7]:
net = caffe.Net(prototxt_path, caffe.TEST)
#net = caffe.Net(prototxt_path,weights_path , caffe.TEST)

Number of image : 654
Separated in 37 classes : {'toilet': 32, 'whiteboard': 29, 'wall': 0, 'night_stand': 31, 'books': 22, 'mirror': 18, 'table': 6, 'chair': 4, 'floor': 1, 'tv': 24, 'lamp': 34, 'window': 8, 'curtain': 15, 'blinds': 12, 'dresser': 16, 'picture': 10, 'ceiling': 21, 'door': 7, 'shower_curtain': 27, 'fridge': 23, 'cabinet': 2, 'sink': 33, 'desk': 13, 'bookshelf': 9, 'towel': 26, 'box': 28, 'floor_mat': 19, 'shelves': 14, 'sofa': 5, 'counter': 11, 'bed': 3, 'person': 30, 'paper': 25, 'bag': 36, 'bathtub': 35, 'pillow': 17, 'clothes': 20}


#### Blobs & Layers

In [8]:
B = [(k, v.data.shape) for k, v in net.blobs.items()]
L = [(k, v[0].data.shape, v[1].data.shape) for k, v in net.params.items() if k != 'upscore37']

In [9]:
B

[('data', (1, 1, 427, 561)),
 ('label', (1, 1, 427, 561)),
 ('data_jg_input_layer_0_split_0', (1, 1, 427, 561)),
 ('data_jg_input_layer_0_split_1', (1, 1, 427, 561)),
 ('label_jg_input_layer_1_split_0', (1, 1, 427, 561)),
 ('label_jg_input_layer_1_split_1', (1, 1, 427, 561)),
 ('conv1_1', (1, 64, 625, 759)),
 ('conv1_2', (1, 64, 625, 759)),
 ('pool1', (1, 64, 313, 380)),
 ('conv2_1', (1, 128, 313, 380)),
 ('conv2_2', (1, 128, 313, 380)),
 ('pool2', (1, 128, 157, 190)),
 ('conv3_1', (1, 256, 157, 190)),
 ('conv3_2', (1, 256, 157, 190)),
 ('conv3_3', (1, 256, 157, 190)),
 ('pool3', (1, 256, 79, 95)),
 ('conv4_1', (1, 512, 79, 95)),
 ('conv4_2', (1, 512, 79, 95)),
 ('conv4_3', (1, 512, 79, 95)),
 ('pool4', (1, 512, 40, 48)),
 ('conv5_1', (1, 512, 40, 48)),
 ('conv5_2', (1, 512, 40, 48)),
 ('conv5_3', (1, 512, 40, 48)),
 ('pool5', (1, 512, 20, 24)),
 ('fc6', (1, 4096, 14, 18)),
 ('fc7', (1, 4096, 14, 18)),
 ('score_fr', (1, 37, 14, 18)),
 ('upscore', (1, 37, 480, 608)),
 ('score', (1, 37, 

#### Updating with new params

In [10]:
CasesGray = {}

#CasesGray['HEB']={}
#CasesGray['HEB']['weights'] = '/data/soft/digits/digits/jobs/20170205-202910-b71d/snapshot_iter_15900.caffemodel'
#CasesGray['HEB']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data/d_raw_histeqBack_8bits/'

#CasesGray['HEB-SO']={}
#CasesGray['HEB-SO']['weights'] = '/data/soft/digits/digits/jobs/20170205-202647-f328/snapshot_iter_15900.caffemodel'
#CasesGray['HEB-SO']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data_spreadout/d_raw_histeqBack_8bits/'

CasesGray['HER-SO']={}
CasesGray['HER-SO']['weights'] = '/data/soft/digits/digits/jobs/20170205-010629-0ab5/snapshot_iter_15900.caffemodel'
CasesGray['HER-SO']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data_spreadout/d_raw_histeqRandom_8bits/'

#CasesGray['HER']={}
#CasesGray['HER']['weights'] = '/data/soft/digits/digits/jobs/20170205-010303-e57c/snapshot_iter_15900.caffemodel'
#CasesGray['HER']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data/d_raw_histeqRandom_8bits/'

#CasesGray['HEF-SO']={}
#CasesGray['HEF-SO']['weights'] = '/data/soft/digits/digits/jobs/20170205-010122-2092/snapshot_iter_15900.caffemodel'
#CasesGray['HEF-SO']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data_spreadout/d_raw_histeqFront_8bits/'

#CasesGray['HEF']={}
#CasesGray['HEF']['weights'] = '/data/soft/digits/digits/jobs/20170205-005712-dc0f/snapshot_iter_15900.caffemodel'
#CasesGray['HEF']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data/d_raw_histeqFront_8bits/'

CasesGray['d-SO']={}
CasesGray['d-SO']['weights'] = '/data/soft/digits/digits/jobs/20170205-005600-ebd9/snapshot_iter_15900.caffemodel'
CasesGray['d-SO']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data_spreadout/d_raw_normal_8bits/'

CasesGray['d']={}
CasesGray['d']['weights'] = '/data/soft/digits/digits/jobs/20170205-005349-cd70/snapshot_iter_15900.caffemodel'
CasesGray['d']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data/d_raw_normal_8bits/'

In [11]:
#Cases = {}

#Cases['HEB-cube-SO']={}
#Cases['HEB-cube-SO']['weights'] = '/data/soft/digits/digits/jobs/20170205-202832-87aa/snapshot_iter_15900.caffemodel'
#Cases['HEB-cube-SO']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data_spreadout/d_raw_histeqBack_cubehelix_8bits/'

#Cases['HEB-jet']={}
#Cases['HEB-jet']['weights'] = '/data/soft/digits/digits/jobs/20170205-202723-8a6d/snapshot_iter_15900.caffemodel'
#Cases['HEB-jet']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data/d_raw_histeqBack_jet_8bits/'

#Cases['HEB-jet-SO']={}
#Cases['HEB-jet-SO']['weights'] = '/data/soft/digits/digits/jobs/20170205-202706-7a73/snapshot_iter_15900.caffemodel'
#Cases['HEB-jet-SO']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data_spreadout/d_raw_histeqBack_jet_8bits/'

#Cases['HER-jet-SO']={}
#Cases['HER-jet-SO']['weights'] = '/data/soft/digits/digits/jobs/20170205-010504-7a95/snapshot_iter_15900.caffemodel'
#Cases['HER-jet-SO']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data_spreadout/d_raw_histeqRandom_jet_8bits/'

#Cases['HER-jet']={}
#Cases['HER-jet']['weights'] = '/data/soft/digits/digits/jobs/20170205-010417-8bfd/snapshot_iter_15900.caffemodel'
#Cases['HER-jet']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data/d_raw_histeqRandom_jet_8bits/'

#Cases['HER-cube-SO']={}
#Cases['HER-cube-SO']['weights'] = '/data/soft/digits/digits/jobs/20170205-010344-23f0/snapshot_iter_15900.caffemodel'
#Cases['HER-cube-SO']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data_spreadout/d_raw_histeqRandom_cubehelix_8bits/'

#Cases['HEF-cube']={}
#Cases['HEF-cube']['weights'] = '/data/soft/digits/digits/jobs/20170205-005813-6c56/snapshot_iter_15900.caffemodel'
#Cases['HEF-cube']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data/d_raw_histeqFront_cubehelix_8bits/'

#Cases['HER-cube']={}
#Cases['HER-cube']['weights'] = '/data/soft/digits/digits/jobs/20170204-042000-d100/snapshot_iter_15900.caffemodel'
#Cases['HER-cube']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data/d_raw_histeqRandom_cubehelix_8bits/'

#Cases['HEF-cube-SO']={}
#Cases['HEF-cube-SO']['weights'] = '/data/soft/digits/digits/jobs/20170204-041941-b4f8/snapshot_iter_15900.caffemodel'
#Cases['HEF-cube-SO']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data_spreadout/d_raw_histeqFront_cubehelix_8bits/'

#Cases['HEF-jet']={}
#Cases['HEF-jet']['weights'] = '/data/soft/digits/digits/jobs/20170204-041858-8e96/snapshot_iter_15900.caffemodel'
#Cases['HEF-jet']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data/d_raw_histeqFront_jet_8bits/'

#Cases['HEF-jet-SO']={}
#Cases['HEF-jet-SO']['weights'] = '/data/soft/digits/digits/jobs/20170204-041843-fd2b/snapshot_iter_15900.caffemodel'
#Cases['HEF-jet-SO']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data_spreadout/d_raw_histeqFront_jet_8bits/'

#Cases['jet-SO']={}
#Cases['jet-SO']['weights'] = '/data/soft/digits/digits/jobs/20170204-041458-dc3f/snapshot_iter_15900.caffemodel'
#Cases['jet-SO']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data_spreadout/d_raw_jet_8bits/'

#Cases['cube-SO']={}
#Cases['cube-SO']['weights'] = '/data/soft/digits/digits/jobs/20170204-041449-00c6/snapshot_iter_15900.caffemodel'
#Cases['cube-SO']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data_spreadout/d_raw_cubehelix_8bits/'

#Cases['jet']={}
#Cases['jet']['weights'] = '/data/soft/digits/digits/jobs/20170204-041429-f757/snapshot_iter_15900.caffemodel'
#Cases['jet']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data/d_raw_jet_8bits/'

#Cases['cube']={}
#Cases['cube']['weights'] = '/data/soft/digits/digits/jobs/20170204-041412-add7/snapshot_iter_15900.caffemodel'
#Cases['cube']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data/d_raw_cubehelix_8bits/'

#Cases['DNA']={}
#Cases['DNA']['weights'] = '/data/soft/digits/digits/jobs/20170204-041401-172d/snapshot_iter_15900.caffemodel'
#Cases['DNA']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data/d_raw_DNA_8bits/'

#Cases['HHA']={}
#Cases['HHA']['weights'] = '/data/soft/digits/digits/jobs/20170204-041220-ff1a/snapshot_iter_15900.caffemodel'
#Cases['HHA']['dataset'] = '/home/jguerry/workspace/datasets/SUNRGBD_pv/data/d_raw_HHA_8bits/'

In [12]:
Cases=CasesGray
Cases.keys()


['HEB-SO', 'HER', 'HEF-SO', 'HEF', 'HER-SO', 'd-SO', 'd']

In [13]:
for case in log_progress(Cases.keys(),title='Model', end_msg='Finished !'): 
        weights_path = Cases[case]['weights']
        net.copy_from(weights_path)
        newParams = {}
        newParams['images_folder']= Cases[case]['dataset']
        newParams['iter_counter']=0
        newParams['countEpoch']=0
        net.layers[0].update(newParams)
        
        results = []
        losses = []
        
        n_cl = net.blobs['score'].channels
        hist = np.zeros((n_cl, n_cl))
        loss = 0
        print("###############################################################")
        print("NEW TESTING : model",case)
        #for idx in log_progress(range(net.layers[0].nb_images), title='iteration', end_msg=str(i),removeAtTheEnd=True):
        for idx in log_progress(range(net.layers[0].nb_images), title='iteration', removeAtTheEnd=True):
            net.forward()
            score_map = net.blobs['score'].data[0].argmax(0)
            gt_map = net.blobs['label'].data[0, 0]
            hist = fast_hist(gt_map.flatten(),score_map.flatten(),n_cl)
            # compute the loss as well
            loss = net.blobs['loss'].data.flat[0]
            results.append(hist)
            losses.append(loss)
        print("Images processed =",net.layers[0].iter_counter)
        print("Cumulative loss =",sum(losses))
        evaluate(results)

###############################################################
NEW TESTING : model HEB-SO
New epoch [total epoch : 1 ].
Images processed = 0
Cumulative loss = 1079.79950511
Accuracy : 0.549918535227
per-class accuracy : 0.210311374616
per-class IU : 0.142535239174
per-class IU (frequency weighted) : 0.363852002631


/data/soft/anaconda2/envs/jg_pfr/lib/python2.7/site-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in true_divide
/data/soft/anaconda2/envs/jg_pfr/lib/python2.7/site-packages/ipykernel/__main__.py:14: RuntimeWarning: invalid value encountered in true_divide


###############################################################
NEW TESTING : model HER
New epoch [total epoch : 1 ].
Images processed = 0
Cumulative loss = 1016.9954049
Accuracy : 0.573116718641
per-class accuracy : 0.265964100606
per-class IU : 0.181113981037
per-class IU (frequency weighted) : 0.397429225716
###############################################################
NEW TESTING : model HEF-SO
New epoch [total epoch : 1 ].
Images processed = 0
Cumulative loss = 1048.33446515
Accuracy : 0.559993614963
per-class accuracy : 0.244005003255
per-class IU : 0.161952194929
per-class IU (frequency weighted) : 0.383826675968
###############################################################
NEW TESTING : model HEF
New epoch [total epoch : 1 ].
Images processed = 0
Cumulative loss = 1032.33075523
Accuracy : 0.570776533158
per-class accuracy : 0.255285276666
per-class IU : 0.172306917805
per-class IU (frequency weighted) : 0.395270318509
########################################################